# DATA CLEANING FOR RELEVANT MRI SEQUENCE

In [8]:
import os
import nibabel as nib
from nibabel.testing import data_path
import numpy as np
import cv2 
from matplotlib import pyplot as plt 
import sys
import shutil

### Load CSV

In [49]:
tesla = 3 #1.5 or 3
downloadDate = "12_05_2022"
path = "./{}T/{}/{}/".format(tesla, seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
# path = "./data/{}/{}/".format(seq, cond)

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

20230204-154554


### Creating List of Relevant Weighted Data
----
The first data cleaning is to generate metadata of relevant MRI sequences from the original metadata.

Relevant T1w MRI dataframe is obtained by including only `MPRAGE` from the `Description` column, and only `TSE/FSE` for T2w MRI. There is no use for filterring out certain visit times on `visit` column (in which such filter was used the first time to easily balance the data size for each group. See the first but unused source code: `data_clean.ipynb`). 

Generated output from this process is saved inside the folder `/TempMeta/` with the file name `Cleaned_Ori_Nolimit_{seq}w_{group}_ADNI1_{tesla}T.csv.csv`


In [73]:
from pathlib import Path
import pandas as pd

def createMetaCombinedString(meta_df):
    meta_image = meta_df.loc[:,"Image Data ID"]
    meta_subject = meta_df.loc[:,"Subject"]
    meta_visit = meta_df.loc[:,"Visit"]

    meta_combined= []
    i=0
    for i in range(len(meta_image)):
        # combined = str(meta_subject[i])+"-"+str(meta_series[i])+"-"+str(meta_image[i])
        combined = str(meta_subject[i])+"-"+str(meta_image[i]) #to check data availability based on subject and image id
        meta_combined.append(combined)
        i+=1

    return meta_combined

def cleanMetadata(meta_df, seq, cond, tesla=3):
    nii_path = "./{}T/{}/{}/".format(tesla, seq, cond)

    target_meta = "./TempMeta/"
    target_path = "./TempData/"
    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    result = list(Path(nii_path).glob('**/*.nii'))
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    ctr = 0

    if seq in "T1":
        desc1 = "RAGE"
        desc2 = "rage"
    elif seq in "T2":
        desc1 = "FSE"
        desc2 = "TSE"

    meta_combined = createMetaCombinedString(meta_df)

    for f in result[:]:
        pathToFile = str(f)
        fileName = pathToFile.split("/")[7] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        # print(fileName)
        
        part = fileNameNoExt.split('_MR')
        id_subject = part[0].split("ADNI_")[1]
        part = part[1].split("raw_")[1]
        id_series = part.split("_S")[1]
        id_series = id_series.split("_I")[0]
        id_image = fileNameNoExt.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        
        if desc1 in fileName or desc2 in fileName: #FILTER BASED ON DESCRIPTION COLUMN FOR RELEVANT SEQUENCE
            target_dir =os.path.join(target_path, seq)
            target_dir =os.path.join(target_dir, cond)
            target_dir = os.path.join(target_dir, subdirName)
            isExist = os.path.exists(target_dir)    

            j = 0
            for j in range(len(meta_combined)):
                if meta_combined[j] in combinedIDs:
                    meta_dict["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict["Format"].append( meta_df.loc[j,"Format"])

                    # if isExist == False:
                    #     os.mkdir(target_dir)
                    # print(target_dir)
                    # print("File {} copied to target folder".format(fileName))
                    # shutil.copy(f,target_dir+"/"+fileName)

                    sim+=1
                j+=1
        ctr+=1    
        # print("File {}".format(ctr))
    print("Total {}w - {} data is {}".format(seq, cond, sim))

    
    return meta_dict


def exportCSV(meta_dict, seq, cond, tesla=3):
# convert the history.history dict to a pandas DataFrame:     
    temp_meta_df = pd.DataFrame(meta_dict) 

    meta_csv_file = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla)

    with open(meta_csv_file, mode='w') as f:
        temp_meta_df.to_csv(f)


In [74]:
dict_1_ad = cleanMetadata(meta_df_1_ad, "T1", "AD")
dict_1_mci = cleanMetadata(meta_df_1_mci, "T1", "MCI")
dict_1_cn = cleanMetadata(meta_df_1_cn, "T1", "CN")
dict_2_ad = cleanMetadata(meta_df_2_ad, "T2", "AD")
dict_2_mci = cleanMetadata(meta_df_2_mci, "T2", "MCI")
dict_2_cn = cleanMetadata(meta_df_2_cn, "T2", "CN")

---------
T1w-AD
Original number: 320
Unique result:320
Total T1w - AD data is 139
---------
T1w-MCI
Original number: 3479
Unique result:3479
Total T1w - MCI data is 993
---------
T1w-CN
Original number: 3382
Unique result:3382
Total T1w - CN data is 887
---------
T2w-AD
Original number: 455
Unique result:455
Total T2w - AD data is 73
---------
T2w-MCI
Original number: 1567
Unique result:1567
Total T2w - MCI data is 220
---------
T2w-CN
Original number: 2082
Unique result:2082
Total T2w - CN data is 153


In [75]:
exportCSV(dict_1_ad, "T1", "AD")
exportCSV(dict_1_mci, "T1", "MCI")
exportCSV(dict_1_cn, "T1", "CN")
exportCSV(dict_2_ad, "T2", "AD")
exportCSV(dict_2_mci, "T2", "MCI")
exportCSV(dict_2_cn, "T2", "CN")
